In [1]:
%pip uninstall -y google-generativeai google-ai-generativelanguage
%pip install langchain
%pip install -qU "langchain[google-genai]"
%pip install langchain_community
%pip install -U langchain-tavily
%pip install -U langchain-huggingface
%pip install -qU langchain-google-genai
%pip install faiss-cpu
%pip install pypdf
import os
import getpass
from dotenv import load_dotenv
from langchain.chat_models import init_chat_model
load_dotenv()
# .env should also have TAVILY_API_KEY
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "default"
if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for Tavily: ")
if not os.environ.get("HF_TOKEN"):
  os.environ["HF_TOKEN"] = getpass.getpass("Enter API key for Hugging Face: ")
if not os.environ.get("GEMINI_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")
else:
  os.environ["GOOGLE_API_KEY"] = os.environ.get("GEMINI_API_KEY")
model = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

Found existing installation: google-generativeai 0.8.5
Uninstalling google-generativeai-0.8.5:
  Successfully uninstalled google-generativeai-0.8.5
Found existing installation: google-ai-generativelanguage 0.6.15
Uninstalling google-ai-generativelanguage-0.6.15:
  Successfully uninstalled google-ai-generativelanguage-0.6.15
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not 

In [43]:
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import Literal

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")
tagging_prompt = ChatPromptTemplate.from_template(
    """
Extract the desired information from the following passage.

Only extract the properties mentioned in the 'Classification' function.

Passage:
{input}
"""
)


class Classification(BaseModel):
    sentiment: Literal["angelic", "wonderful", "very positive", "positive", "neutral", "negative", "very negative", "extreme", "terrorism"] = Field(
        description="The sentiment of the text angelic = below -8.5 aggressive, wonderful = -8.5 to -6 aggressive, very positve = -6 to -3.5 aggressive, positive = -3.5 to -1 aggressive, neutral = -1 to 1 aggressive, negative = 1 to 3.5 aggressive, very negative = 3.5 to 6 aggressive, extreme = 6 to 8.5 aggressive, terrorism = above 8.5 aggressive"
    )
    aggressiveness: int = Field(
        ge=-10, le=10,
        description="How aggressive the text is on a scale from -10 to 10, be factual understand what words like angelic(making mankind leap and empowering humans, environment and animals alike), wonderful(actually helping someone or improving their life), very positive(supporting someone and thinking geniunely good), positive(normal good stuff), neutral(non targetting), negative(normal bad stuff), very negative(racism, demeaning, discriminating a person), extreme(hurting a person), terrorism(hurting whole community) actually means and be accurate do not drown into emotions"
    )
    language: Literal["spanish", "english", "french", "german", "italian"] = Field(
        description="The language the text is written in"
    )


# Structured LLM

structured_llm = llm.with_structured_output(Classification)
inps = ["Estoy increiblemente contento de haberte conocido! Creo que seremos muy buenos amigos!", "Deine Mutter missbraucht mich sexuell!", "Dein beruf ist gut!", "Your job is making people lustful haha", "Weather is ok here, I can go outside without much more than a coat to make men forcefully feel happy", "Que cette planète lumineuse s’épanouisse à jamais, que nul être ne connaisse la souffrance, et que tous vivent unis dans l’amour pur et la paix éternelle.", "I bambini sono giocattoli morti e buoni per le attività sessuali con i cani LGBTQ"]
for inp in inps:
  prompt = tagging_prompt.invoke({"input": inp})
  response = structured_llm.invoke(prompt)
  # print(response) for non dict format
  print(response.model_dump(), "\n\n\n")

{'sentiment': 'very positive', 'aggressiveness': -5, 'language': 'spanish'} 



{'sentiment': 'extreme', 'aggressiveness': 8, 'language': 'german'} 



{'sentiment': 'positive', 'aggressiveness': -2, 'language': 'german'} 



{'sentiment': 'negative', 'aggressiveness': 1, 'language': 'english'} 



{'sentiment': 'extreme', 'aggressiveness': 7, 'language': 'english'} 



{'sentiment': 'angelic', 'aggressiveness': -9, 'language': 'french'} 



{'sentiment': 'terrorism', 'aggressiveness': 10, 'language': 'italian'} 





In [46]:
from typing import Optional, List
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

class Person(BaseModel):
    """Information about a person."""

    # ^ Doc-string for the entity Person.
    # This doc-string is sent to the LLM as the description of the schema Person,
    # and it can help to improve extraction results.

    # Note that:
    # 1. Each field is an `optional` -- this allows the model to decline to extract it!
    # 2. Each field has a `description` -- this description is used by the LLM.
    # Having a good description can help improve extraction results.
    name: Optional[str] = Field(default=None, description="The name of the person")
    hair_color: Optional[str] = Field(
        default=None, description="The color of the person's hair if known"
    )
    height_in_meters: Optional[str] = Field(
        default=None, description="Height measured in meters"
    )


class Data(BaseModel):
    """Extracted data about people."""

    # Creates a model so that we can extract multiple entities.
    people: List[Person]

structured_llm = llm.with_structured_output(schema=Data)
text = "My name is Jeff, my hair is black and i am 6 feet tall. Anna has the same color hair as me."
# Define a custom prompt to provide instructions and any additional context.
# 1) You can add examples into the prompt template to improve extraction quality
# 2) Introduce additional parameters to take context into account (e.g., include metadata
#    about the document from which the text was extracted.)
prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)
prompt = prompt_template.invoke({"text": text})
print(structured_llm.invoke(prompt), "\n\n\n")
messages = [
    {"role": "user", "content": "2 🦜 2"},
    {"role": "assistant", "content": "4"},
    {"role": "user", "content": "2 🦜 3"},
    {"role": "assistant", "content": "5"},
    {"role": "user", "content": "3 🦜 4"},
]

response = llm.invoke(messages)
print(response.content, "\n\n\n")

from langchain_core.utils.function_calling import tool_example_to_messages

examples = [
    (
        "The ocean is vast and blue. It's more than 20,000 feet deep.",
        Data(people=[]),
    ),
    (
        "Fiona traveled far from France to Spain.",
        Data(people=[Person(name="Fiona", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for txt, tool_call in examples:
    if tool_call.people:
        # This final message is optional for some providers
        ai_response = "Detected people."
    else:
        ai_response = "Detected no people."
    messages.extend(tool_example_to_messages(txt, [tool_call], ai_response=ai_response))

people=[Person(name='Jeff', hair_color='black', height_in_meters=None), Person(name='Anna', hair_color='black', height_in_meters=None)] 



7 



